Copyright (c) 2023 Graphcore Ltd. All rights reserved.

# Object Detection with YOLO v4 model executed on IPU
This notebook demonstrates the object detection task realized using YOLOv4 model inference pipeline run on Graphcore IPUs. Originally, the code of [YOLOv4 model adapted to IPU was published in examples github repository](https://github.com/graphcore/examples/tree/master/vision/yolo_v4/pytorch).

### Summary table

|  Domain | Tasks | Model | Datasets | Workflow |   Number of IPUs   | Execution time |
|---------|-------|-------|----------|----------|--------------|--------------|
| vision  | object detection | YOLO v4 | COCO | inference | recommended: 1 (min: 4) | 2mn    |


![object detection on IPU](notebook/first_example.png "Last supper object detection")


## Environment setup

The best way to run this demo is on Paperspace Gradient's cloud IPUs because everything is already set up for you. To run the demo using other IPU hardware, you need to have the Poplar SDK enabled and the relevant PopTorch wheels installed. Refer to the [getting started guide](https://docs.graphcore.ai/en/latest/getting-started.html#getting-started) for your system for details on how to enable the Poplar SDK and install the PopTorch wheels.


## Requirements
Before using the model on IPU you have to build the custom operations for IPU:

In [ ]:
!make

and install the Python dependencies:

In [ ]:
%pip install -r requirements.txt

## COCO dataset

This demonstration example of inference with YOLO v4 model is using the checkpoint of model trained with [COCO dataset](https://cocodataset.org/). Therefore, we can demonstrate detection of 80 different classes.

In [ ]:
from ruamel import yaml

class_names = yaml.safe_load(open("configs/class_name.yaml"))["class_names"]
class_names

## Model preparation

In [ ]:
import os
import time
import torch

from PIL import Image
from pathlib import Path
from models.yolov4_p5 import Yolov4P5
from notebook.yolo_ipu_pipeline import YOLOv4InferencePipeline

import poptorch

poptorch.setLogLevel(4)  # ERR log level

path_to_detection = Path().parent.resolve()
os.environ["PYTORCH_APPS_DETECTION_PATH"] = str(path_to_detection)

We use the original YOLOv4 model with checkpoint trained with COCO dataset, which we pass as `checkpoint`.

In [ ]:
model = Yolov4P5
checkpoint = "checkpoint/yolov4_p5_reference_weights/yolov4-p5-sd.pt"

In [ ]:
%%bash

FILE=./checkpoint/yolov4_p5_reference_weights/yolov4-p5-sd.pt; \
if [ -f "$FILE" ]; then \
    echo "$FILE exists, no need to download."; \
else \
    mkdir checkpoint; \
    cd checkpoint; \
    curl https://gc-demo-resources.s3.us-west-1.amazonaws.com/yolov4_p5_reference_weights.tar.gz -o yolov4_p5_reference_weights.tar.gz && tar -zxvf yolov4_p5_reference_weights.tar.gz && rm yolov4_p5_reference_weights.tar.gz; \
    cd ..; \
fi 

We use a `YOLOv4InferencePipeline` class to set all IPU specific options and wrap the PyTorch model into Graphcore PopTorch inference model. The pipeline reads the configuration parameters from the [config file](configs/override-inference-yolov4p5.yaml).

In [ ]:
pipeline = YOLOv4InferencePipeline(
    model=Yolov4P5,
    checkpoint_path="checkpoint/yolov4_p5_reference_weights/yolov4-p5-sd.pt",
)

We demonstrate the inference on an exemplary image of famous painting, which is stored in the repository.

In [ ]:
image = Image.open("notebook/last_supper_restored.jpeg")

The pipeline call consists of preprocessing, forward pass of the preprocessed image through the model and model output postprocessing.
When used for the first time, a one-time compilation of the model is triggered.
For a single IPU it should take about 3 min for the first time.

In [ ]:
start_time = time.time()

processed_batch = pipeline(image)

inference_time = time.time() - start_time
print("pipeline inference time: ", inference_time * 10e3, "msec")

You can now check what exactly has been detected on the input image.
The output contains a list of detected objects for each image in batch.
The result contain five numbers: first four are the coordinates in XYWH format (x, y, width, height of bounding box), and the fifth number represent the predicted class of detected object.

In [ ]:
keys = ["x", "y", "width", "height", "class"]
objects_detected = []
for obj in processed_batch[0][0].tolist():
    objects_detected.append({key: value for key, value in zip(keys, obj)})
    objects_detected[-1]["class"] = class_names[int(objects_detected[-1]["class"])]

objects_detected

Also you can use the coordinates for the detected objects to plot the bounding boxes on the original image.

In [ ]:
img_paths = pipeline.plotImg(processed_batch, image)
img_paths

## Try it on your own

Let's check the YOLOv4 model on IPU with some image from the internet.

We may use wget command to store the image as `image.png`. The syntax is `!wget -O image.png [YOUR URL]`, for example:

In [ ]:
# UPLOAD YOUR OWN IMAGE HERE BY REPLACING THE URL

!wget -O image.png https://www.graphcore.ai/hubfs/assets/images/content/new-team.jpg

Let's now use this image for inference with the same model.

In [ ]:
start_time = time.time()

image = Image.open("image.png")

processed_batch = pipeline(image)
img_paths = pipeline.plotImg(processed_batch, image)

total_time = time.time() - start_time
print("Total time until plot: ", total_time, "sec")

### Optional - Release IPUs in use

The IPython kernel has a lock on the IPUs used in running the model, preventing other users from using them. For example, if you wish to use other notebooks after working your way through this one, it may be necessary to manually run the below cell to release IPUs from use. This will happen by default if using the Run All option. More information on the topic can be found at [Managing IPU Resources](https://github.com/gradient-ai/Graphcore-HuggingFace/blob/main/useful-tips/managing_ipu_resources.ipynb).

In [ ]:
pipeline.detach()